In [5]:
import copy
# TODO: fix die-adaptive lr, show usefull stat instead of logs
# "batch norm analog" - i.e. collector neuron - when things will go slow with reasonable dataset - likely cifar10, then see if speed improvement
# TODO: 
#  1. investigate living dangle Neurons
#  2. investigate gradient size (bigger than lr? limit to lr)
#  3. Collect stats on gradients, neuron values, over training step
#  4. Fix scrolling
#  5. Use Cuda
#  6a. Relu -> Relu6
#  6b. Improve death, introduce train mode/ death mode (kill any which reach zero)
# After that can think on Cifar10 convnet death

import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
from ai_libs.basic.deep_reload import deep_reload
import livenet
deep_reload(livenet)
import ai_libs.simple_log as simple_log
from ai_libs.simple_log import LOG
simple_log.level = simple_log.LogLevel.INFO
import math
from matplotlib import pyplot as plt
plt.ion()
%matplotlib TkAgg
livenet.core.utils.set_seed()
var = 42
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



2.3.1+cu121


'%.4f'

In [2]:
downscale = (2, 2)
test = livenet.datasets.get_mnist_test()
train = livenet.datasets.get_mnist_train()
test_x, test_y = livenet.datasets.to_plain(*test, downscale=downscale, to_gray=True, to_odd=True)
train_x, train_y = livenet.datasets.to_plain(*train, downscale=downscale, to_gray=True, to_odd=True)


In [33]:

train_x, train_y = livenet.datasets.get_odd_2()
test_x, test_y = train_x, train_y


In [3]:
network = livenet.nets.create_perceptron(train_x.shape[1], 1, 2)
batch_iterator = livenet.gen_utils.batch_iterator(train_x, train_y, batch_size=3)
criterion = livenet.nets.criterion_classification_n
optimizer = livenet.nets.create_optimizer(network)
optimizer.learning_rate = 0.01
trainer = livenet.net_trainer.NetTrainer(network, batch_iterator, criterion, optimizer, epoch_size=1)

Iˈ0.000 S0 became useless livenet/core/death.py:68
Iˈ0.001 D0 became dangle livenet/core/death.py:59
Iˈ0.003 D1 became dangle livenet/core/death.py:59
Iˈ0.003 N0 became useless livenet/core/death.py:68
Iˈ0.004 N0 became dangle livenet/core/death.py:59
LiveNet


In [4]:
with torch.no_grad():
    network.inputs[0].axons[0].destination.b[...] = 200
    network.inputs[0].axons[0].destination.axons[0].k[...] = -10.2
    network.inputs[0].axons[0].destination.axons[1].k[...] = 10.1

train_x[...] = 0.0


#network.inputs[0].axons[0].destination.axons[1].k[...]
#network.inputs[0].axons[0].destination.b[...]
#network.inputs[0].axons[0].destination.axons[0].k[]



In [5]:
# simple_log.level = simple_log.LogLevel.DEBUG
network.context.regularization_l1 = 0.001
optimizer.learning_rate = 0.01
trainer.step(3500)


Iˈ0.000 3904.895 = 3904.895+0.000reg params=6 livenet/net_trainer.py:93


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [82]:
pred = network(test_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = test_y.numpy() - pred_bin
f"accuracy {len(diff[diff == 0]) / len(diff)}"


'accuracy 0.6666666666666666'

In [98]:
%matplotlib
plt.ion()

def param_amount(param):
    val = len(param["params"])
    return val

def param_max(param):
    val = np.max(np.abs(param["params"]))
    return val

def param_picker1(param):
    try:
        val0 = param["params"]["N0->D0"].item()
        # val0 = param["loss"]
    except KeyError:
        val0 = 0.
    return val0

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

#plt.figure(figsize=(16, 9))
plt.grid()
values = get_param_values(trainer.history, param_picker1)
plt.plot(values)
# accum = core.stat_utils.AccumStat()
# accum.add_value(trainer.network.parameters())
# accum.plot()


Using matplotlib backend: tkagg


In [96]:
print(network.context.health_stat.get_stat())
for name, p in sorted(network.named_parameters(),key=lambda x: len(x[0]) + ord(x[0][0]) / 1000. + ord(x[0][-1]) / 10000):
    print(name, f"{p.item():.2f}")


{'dangle': {'RegularNeuron': 1, 'DestinationNeuron': 0}, 'useless': {'RegularNeuron': 0, 'DataNeuron': 1}}
D0 -30.38
D1 0.00
N0 20.00
N0->D0 1.55
N0->D1 0.00


In [191]:
[x.name for x in network.context.health_stat.dangle]

['N3', 'N11']

In [92]:
def build_symmetric_dangle_net():
    from livenet.core.livenet import DestinationNeuron, RegularNeuron, DataNeuron
    net = livenet.core.livenet.LiveNet()
    net.outputs += [DestinationNeuron(net.context, activation=None), DestinationNeuron(net.context, activation=None)]
    net.inputs += [DataNeuron(net.context)]
    neuron = RegularNeuron(net.context, activation=torch.nn.ReLU())
    neuron.connect_to(net.outputs[0])
    neuron.connect_to(net.outputs[1])
    # net.inputs[0].connect_to(neuron)
    net.root.visit("init_weight")
    with torch.no_grad():
        net.outputs[0].b[...] = -30.0
        #net.outputs[0].b.requires_grad = False
        net.outputs[1].b[...] = -0
        net.outputs[1].b.requires_grad = False
        neuron.b[...] = 20.0
        neuron.b.requires_grad = False
        neuron.axons[0].k[...] = 2.0
        # neuron.axons[0].k.requires_grad = False
        neuron.axons[1].k[...] = -0
        neuron.axons[1].k.requires_grad = False
    return net

train_x, train_y = livenet.datasets.get_odd_2()  
test_x, test_y = train_x, train_y
# train_x is not actually used



In [93]:
network = build_symmetric_dangle_net()
# train_x is not actually used as networks input doesn't connected to anything
batch_iterator = livenet.gen_utils.batch_iterator(train_x, train_y, batch_size=3)
criterion = livenet.nets.criterion_classification_n
optimizer = livenet.nets.create_optimizer(network)
trainer = livenet.net_trainer.NetTrainer(network, batch_iterator, criterion, optimizer, epoch_size=1)


Iˈ0.000 D0 became dangle livenet/core/death.py:59
Iˈ0.001 D1 became dangle livenet/core/death.py:59
Iˈ0.001 S0 became useless livenet/core/death.py:68
Iˈ0.001 N0 became useless livenet/core/death.py:68
Iˈ0.001 N0 became dangle livenet/core/death.py:59
LiveNet


In [97]:
network.context.regularization_l1 = 0.001
optimizer.learning_rate = 0.01
trainer.step(5000)


Iˈ0.000 0.920 = 0.920+0.000reg params=5 livenet/net_trainer.py:93
Iˈ0.003 0.920 = 0.920+0.000reg params=5 livenet/net_trainer.py:93
Iˈ0.007 0.920 = 0.920+0.000reg params=5 livenet/net_trainer.py:93
Iˈ0.010 0.920 = 0.920+0.000reg params=5 livenet/net_trainer.py:93
Iˈ0.015 0.920 = 0.920+0.000reg params=5 livenet/net_trainer.py:93
Iˈ0.019 0.920 = 0.920+0.000reg params=5 livenet/net_trainer.py:93
Iˈ0.024 0.920 = 0.920+0.000reg params=5 livenet/net_trainer.py:93
Iˈ0.030 0.920 = 0.920+0.000reg params=5 livenet/net_trainer.py:93
Iˈ0.035 0.920 = 0.920+0.000reg params=5 livenet/net_trainer.py:93
Iˈ0.041 0.920 = 0.920+0.000reg params=5 livenet/net_trainer.py:93
Iˈ0.047 0.920 = 0.920+0.000reg params=5 livenet/net_trainer.py:93
Iˈ0.052 0.920 = 0.920+0.000reg params=5 livenet/net_trainer.py:93
Iˈ0.061 0.920 = 0.920+0.000reg params=5 livenet/net_trainer.py:93
Iˈ0.069 0.920 = 0.920+0.000reg params=5 livenet/net_trainer.py:93
Iˈ0.077 0.920 = 0.920+0.000reg params=5 livenet/net_trainer.py:93
Iˈ0.086 0.